In [ ]:
!pip install telebot

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 242.9/242.9 kB 4.0 MB/s eta 0:00:00


In [ ]:
import telebot
from telebot import types
from tensorflow.keras.models import load_model
import tensorflow as tf
from tensorflow.keras.preprocessing.sequence import pad_sequences
import numpy as np
import pickle
import pandas as pd
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, GlobalAveragePooling1D, Dropout
from sklearn.preprocessing import LabelEncoder

bot = telebot.TeleBot('6666746743:AAEn47TK-QVf2bqAHwvT0rr5aORzCdQQTJI')

markup = types.ReplyKeyboardMarkup(row_width=2)
itembtn1 = types.KeyboardButton('/recipes')
itembtn2 = types.KeyboardButton('/about_me')
markup.add(itembtn1, itembtn2)

#model = load_model('model.h5')
model=load_model('model.keras')
max_len_ingredients = pickle.load(open('max_len_ingredients.pkl', 'rb'))
tokenizer = pickle.load(open('tokenizer.pkl', 'rb'))
label_encoder = pickle.load(open('label_encoder.pkl', 'rb'))
#max_len_ingredients = 0
@bot.message_handler(commands=['start'])
def main(message):
    bot.send_message(message.chat.id,'Привет, введите /recipes, для начала работы.')


@bot.message_handler(commands=['recipes'])
def main(message):
    bot.send_message(message.chat.id,f'Привет, {message.from_user.first_name}. Введите ингредиенты через запятую:')
    bot.register_next_step_handler(message, process_ingredients)
def process_ingredients(message):
    ingredients = message.text
    max_len_ingredients = len(ingredients)
    predicted_url = predict_recipe_url(ingredients)
    bot.send_message(message.chat.id, 'Ингредиенты: ' + ingredients)
    bot.send_message(message.chat.id, 'Ваш рецепт: ' + predicted_url)
    bot.send_message(message.chat.id, f'{message.from_user.first_name}, удачи в приготовлении изысканного блюда.', reply_markup=markup)
@bot.message_handler(commands=['about_me'])
def main(message):
  bot.send_message(message.chat.id, f'Нажмите или напишите /recipes, чтобы увидеть мой функционал.')


def predict_recipe_url(ingredients):
    ingredients_seq = tokenizer.texts_to_sequences([ingredients])
    ingredients_seq = pad_sequences(ingredients_seq, maxlen=max_len_ingredients, padding='post')
    prediction = model.predict(ingredients_seq)
    predicted_url = label_encoder.inverse_transform([np.argmax(prediction)])
    return predicted_url[0]

bot.polling(none_stop=True)

1/1 [==============================] - 0s 32ms/step
